In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import community

In [ ]:
# Load the edge list and create a directed Graph
with open("hamster.edgelist", 'rb') as fh:
    G = nx.read_edgelist(fh, create_using=nx.DiGraph())

In [ ]:
# Visualize a pandas DataFrame
def visualize(df, name="graph"):
    print('Pagerank distribution')
    df['pagerank'].plot.hist(bins=100)
    plt.title('PageRank Distribution')
    plt.savefig(name + ".png", bbox_inches='tight')
    plt.show()
    
    print('Log scale')
    df['pagerank'].apply(np.log).plot.hist(bins=100)
    plt.title('PageRank Distribution (log scale)')
    plt.savefig(name + "_log.png", bbox_inches='tight')
    plt.show()

In [ ]:
# Sort a pandas DataFrame by PageRank
def sort_by_pagerank(df):
    df = df.sort_values(by='pagerank', ascending=False).reset_index().rename_axis('rank').reset_index().set_index('id')
    df['rank'] += 1
    return df

In [ ]:
# Calculate the PageRank for the nodes in directed graph G
def pagerank(G):
    return sort_by_pagerank(pd.DataFrame.from_dict(nx.pagerank(G), orient='index').rename(columns={0: 'pagerank'}).rename_axis('id'))

In [ ]:
# Display some PageRank statistics from a pandas DataFrame
def display_rank(df):
    df = sort_by_pagerank(df)
    
    print("Top 10:\n")
    display(df.head(10))
    
    print("Lowest 10:\n")
    display(df.tail(10))

In [ ]:
def rank_error(df):
    return df.apply(lambda r: abs(r['rank'] - r['rank_base'])/r['rank_base'], axis='columns').sum()

In [ ]:
def pagerank_error(df):
    return df.apply(lambda r: abs(r['pagerank'] - r['pagerank_base'])/r['rank_base'], axis='columns').sum()

In [ ]:
def errors_of(i, df):
    df = df.rename(columns={'pagerank_{0}'.format(i):'pagerank','rank_{0}'.format(i):'rank'})
    return (rank_error(df), pagerank_error(df))

In [ ]:
def display_errors(df):
    display("Rank error: {0}".format(rank_error(df)))
    display("Pagerank error: {0}".format(pagerank_error(df)))

In [ ]:
# Retrieve the nodes that have a certain in-degree and out-degree
def get_nodes_by_degree(graph, in_degree, out_degree):
    # Initialize a list to store the nodes in
    nodes = []

    # Loop through all the nodes and the degrees
    for node in graph.nodes():
        if graph.in_degree(node) == in_degree and graph.out_degree(node) == out_degree:
            nodes.append(node)

    return nodes

In [ ]:
# Get the leaf nodes of a graph
def get_leaves(graph):
    return get_nodes_by_degree(graph, 1, 0)

In [ ]:
# Retrieve some statistics per node
def get_node_statistics(graph):
    # Initialize a list to store the nodes in
    nodes = []

    for node in graph.nodes():
        in_degree = graph.in_degree(node)
        out_degree = graph.out_degree(node)

        nodes.append({
            'node': node,
            'in_degree': in_degree,
            'out_degree': out_degree,
            'sum': in_degree + out_degree
        })
        
    return pd.DataFrame.from_records(nodes, index='node')

#### Original graph

In [ ]:
df = pagerank(G)
visualize(df, "base")
df = df.rename(columns={'rank':'rank_base','pagerank':'pagerank_base'})

In [ ]:
random.seed(420) # ensure output is deterministic

#### Graph with random edges removed

In [ ]:
Gx = G.copy()
Gx.remove_edges_from(random.sample(G.edges(), 20*G.number_of_edges()//100))
pr = pagerank(Gx)
visualize(pr, "random_edges_removed")
df = pd.merge(df, pr, left_index=True, right_index=True, how='outer').sort_values(by='rank_base')
display_errors(df)
df = df.rename(columns={'pagerank':'pagerank_1','rank':'rank_1'})

#### Graph with random leaves (and their edges) removed

In [ ]:
# Create a copy of the original graph
Gx = G.copy()

# Randomly remove leaves from the graph
leaves = get_leaves(Gx)
leaf_edges = Gx.in_edges(leaves)

# Remove edges only
Gx.remove_edges_from(random.sample(list(leaf_edges), 20*len(leaf_edges)//100))

# Remove both edges and nodes
# Gx.remove_nodes_from(random.sample(leaves, 20*len(leaves)//100))

# Calculate the PageRank
pr = pagerank(Gx)

# Show some statistics about the new graph
visualize(pr, "random_leaves_removed")
df = pd.merge(df, pr, left_index=True, right_index=True, how='outer').sort_values(by='rank_base')
display_errors(df)
df = df.rename(columns={'pagerank':'pagerank_2','rank':'rank_2'})

#### Graph with most connected nodes removed

In [ ]:
# Create a copy of the original graph
Gx = G.copy()

# Sort list in descending order
connected = get_node_statistics(Gx).sort_values(['sum'], ascending=False)

# Select the x most connected nodes, where x is random
most_connected = connected.head(random.randint(1, 20*len(connected)//100))

# Remove some of the most connected nodes
Gx.remove_nodes_from(most_connected.index.tolist())

# Calculate the PageRank
pr = pagerank(Gx)

# Show some statistics about the new graph
visualize(pr)
df = pd.merge(df, pr, left_index=True, right_index=True, how='outer').sort_values(by='rank_base')
display_errors(df)
df = df.rename(columns={'pagerank':'pagerank_3','rank':'rank_3'})

In [ ]:
display(df.head(30), df.tail(30))

### Node Statistics Lookup:

In [ ]:
# Create a copy of the original graph
Gx = G.copy()

stats = get_node_statistics(Gx)
pageranks = pagerank(Gx)

node = 2426

display(stats.iloc[node-1])
display(pageranks[pageranks.index == str(node)])

display(pageranks.head(10))
display(pageranks.tail(10))

In [ ]:
Gx = G.copy().to_undirected()

partition = community.best_partition(Gx)

size = float(len(set(partition.values())))
pos = nx.spring_layout(Gx)
count = 0.
for com in set(partition.values()):
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(Gx, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(Gx,pos, alpha=0.5)
plt.show()